# Simplified Ikeda regression

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

from rolldecayestimators.simplified_ikeda import calculate_roll_damping
from rolldecayestimators import equations
import sympy as sp
from rolldecayestimators import symbols
from rolldecayestimators.substitute_dynamic_symbols import lambdify
from mdldb.tables import Run
from rolldecayestimators.ikeda_estimator import IkedaEstimator
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from sklearn.pipeline import Pipeline

from rolldecay import database

import data
import copy
from rolldecay import database
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

In [ ]:
df_ikeda = database.load('rolldecay_simplified_ikeda',limit_score=0.5)
mask = (df_ikeda['ship_speed']==0)   ## Zero speed!
df_ikeda = df_ikeda.loc[mask].copy()

In [ ]:
df_ikeda.describe()

In [ ]:
df_ikeda['score'].hist(bins=30)

In [ ]:
df_linear = database.load('rolldecay_linear',limit_score=0.8)
mask = (df_linear['ship_speed']==0)   ## Zero speed!
df_linear = df_linear.loc[mask].copy()

In [ ]:
df_linear['score'].hist(bins=30)

In [ ]:
df_compare = pd.merge(left=df_ikeda, right=df_linear, how='left', left_index=True, right_index=True,
                     suffixes=('_ikeda',''))

In [ ]:
df_compare.plot(x='zeta', y='zeta_ikeda', style='.', alpha=0.5)

In [ ]:
fig,ax=plt.subplots()
N=20
bins = np.linspace(df_compare['zeta_ikeda'].min(),df_compare['zeta_ikeda'].max(),N)
df_compare['zeta'].hist(bins=bins, ax=ax, label='linear')
df_compare['zeta_ikeda'].hist(bins=bins, ax=ax, label='linear', alpha=0.5)
